# Hackathon Dotz

### Importando Bibliotecas

In [0]:
import pandas as pd
import numpy as np
import collections
import tqdm
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')
from google.colab import files  


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Importando Arquivos

In [0]:
#Carregar os dados para o google colab
uploaded = files.upload()

In [0]:
# Importação dos dataset
df = pd.read_csv('Hackathon_Base_Treino_comdep.csv')
df2 = pd.read_csv('Hackathon_Base_Teste.csv')

In [0]:
df.head()

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA
2,AGUA MIN SCHIN S/GAS 500ML,SEM GÁS,ÁGUA MINERAL,BEBIDAS NÃO ALCOÓLICAS
3,FITA DUPLA FACE C/SUPORTE SCOTCH,FITA ADESIVA,PAPELARIA,PAPELARIA
4,MASSA PIZZA ROMANHA OREGANO PCT 160G,PIZZA REGULAR,MASSA FRESCA,MASSAS FRESCAS


In [0]:
# Verificando os tamanhos das classificações
classifica = df[['DEPARTAMENTO']]
qtd_dep = int(classifica.nunique().values)
classifica = df[['CATEGORIA']]
qtd_cat = int(classifica.nunique().values)
classifica = df[['SUB-CATEGORIA']]
qtd_sub = int(classifica.nunique().values)

# Conferindo valores
print('Existem', qtd_dep, 'DEPARTAMENTOS')
print('Existem', qtd_cat, 'CATEGORIA')
print('Existem', qtd_sub, 'SUB-CATEGORIA')

# Variáveis de preparação dos dados
pontuacao = ['.',',','-','+',':',';','&','+','/','!','#','%','(',')','  ']
stop_words = ['em','sao','ao','de','da','do','com','para','c','kg','un','ml','pct','und','das',
              'no','ou','sem','gfa','jg','p/','la','1','2','3','4','5','6','7','8','9','0','a',
              'b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v',
              'x','w','y','z','kit','po','liq','cr','sh','pt','des','imp','lt','vd','az','esm',
              'cx','bc','lv','ad','gf','mix','rf','fr','bj','int','cd','cz','tp','ap','rs','caixa',
              'sao','bom','esp','extra','premium','pto','uso','sol','garrafa','brasil','pc','conj',
              'ph','gold','cj','mais','cor','facil','ice','classic','br','vm','tt','em','far','fix',
              'sc','dr','aco','nadir','romana','bauducco','original','pe','peso','sem','pro','sl',
              'und','seda','tradicional','arcor','teka','bas','dec','fina','palmolive','neutro',
              'sec','peq','baw','brinox','ext','fd','up','waw','seara','sadia','del','oetker',
              'royal','ret','boni','black','ln','new','preta','corttex','blue','big','ou','co','ex',
              'paf','pif','box','bdj','bestway','cinza','dourada','gluten','plasvale','protex','ype',
              'fs','for','no','das','lg','me','pvc','refil','tirolez','sm','friboi','ipa','pr','st',
              'xg','xxg','mc','nr','cp','ox','pa','pq','me','unid','ana','bi','di','gr']
stop_words_sub = ['em','sao','ao','de','da','do','com','para','c','kg','un','ml','pct','und','das',
              'no','ou','sem','gfa','jg','p/','la','1','2','3','4','5','6','7','8','9','0']

# Variável tamando da base de teste
test_size = 0.1

# Variáveis do modelo
epochs = 12
batch_size = 128

Existem 53 DEPARTAMENTOS
Existem 332 CATEGORIA
Existem 1968 SUB-CATEGORIA


### Preparando Dados

In [0]:
# Excluindo da descrição texto após os números, informações julgadas irrelevantes para a classificação.
df['DESC_AJUSTADA'] = df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)
df2['DESC_AJUSTADA'] = df2['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')

# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  new_text = TreebankWordDetokenizer().detokenize(stop)
  return new_text

# Eliminando as stop words
df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].apply(tokenize_df)
df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].apply(tokenize_df)

In [0]:
df.head(2)

,DESCRIÇÃO PARCEIRO,SUB-CATEGORIA,CATEGORIA,DEPARTAMENTO,DESC_AJUSTADA
0,"PASTA INT VITAPOWER 1,005KG AMEND/SHOT",TRADICIONAL,CREME DE AMENDOIM,MERCEARIA DOCE,pasta vitapower
1,ESPONJA BETTANIN BRILHUS C/1,MULTIUSO,ESPONJA SINTÉTICA,CUIDADOS COM A COZINHA,esponja bettanin brilhus


In [0]:
df2.head(2)

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,sal rosa himalaia alho
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,bola natal acasa


### Etapa 1 - DEPARTAMENTO

In [0]:
# Selecionando as unicas palavras da DESCRIÇÃO PARCEIRO
texto = df[['DESC_AJUSTADA']]
texto['TOKENIZED'] = texto['DESC_AJUSTADA'].apply(word_tokenize)
texto = list(texto.TOKENIZED)
lista = [item for sublista in texto for item in sublista]
lista = sorted(lista)
unicas = set(lista)
print('Quantidade de observações:', len(texto), '\n')
print('Quantidade de GERAL palavras:', len(lista), '\n')
print('Quantidade de UNICA palavras:', len(unicas), '\n')
counter = collections.Counter(lista)

Quantidade de observações: 22009 

Quantidade de GERAL palavras: 78971 

Quantidade de UNICA palavras: 12101 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#departamentos = list(set(list(df['DEPARTAMENTO'])))
#encoder_dep = tfds.features.text.SubwordTextEncoder(vocab_list=departamentos)
#print('Vocabulary size(DEPARTAMENTO): {}'.format(encoder_dep.vocab_size))

In [0]:
# Gerar um arquivo com a lista de palavras unicas
#arquivo = open('dotz.txt', 'w')
#arquivo.write(str(counter))
#arquivo.close()

In [0]:
# Gerando encoder com o vocabulário das palavras
encoder = tfds.features.text.SubwordTextEncoder(vocab_list=unicas)
print('Vocabulary size(DESCRIÇÃO PARCEIRO): {}'.format(encoder.vocab_size))

Vocabulary size(DESCRIÇÃO PARCEIRO): 12358


In [0]:
# Exemplo do encode e decode da DESCRIÇÃO PARCEIRO	
sample_string = 'pasta int vitapower amend shot'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [5065, 12134, 2267, 12218, 12134, 7379, 12134, 5937, 12134, 5415]
The original string: "pasta int vitapower amend shot"


In [0]:
# Validando a string original com a string após o decode
assert original_string == sample_string

In [0]:
# Exemplificando a relação de index ----> word
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

  # Encode departamado
dep = df['DEPARTAMENTO']
dep_encode = LabelEncoder()
dep_encode.fit_transform(dep)

5065 ----> pasta
12134 ---->  
2267 ----> in
12218 ----> t
12134 ---->  
7379 ----> vitapower
12134 ---->  
5937 ----> amend
12134 ---->  
5415 ----> shot


array([39, 11,  3, ..., 15, 39, 28])

In [0]:
# Definindo feature e target
x = df['DESC_AJUSTADA']
y = dep_encode.fit_transform(dep)

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)

In [0]:
# Criação da matriz
def pad_to_size(vec, size):
  zeros = [0] * (size - len(vec))
  vec.extend(zeros)
  return vec

# Encode do input
def encode_input(descricao):
  lista_x = []
  for desc in descricao:
    desc_encode = encoder.encode(desc)
    desc_encode = pad_to_size(desc_encode, 64)
    lista_x.append(desc_encode)

  # Convertendo x em tensor
  input_encode = tf.cast(lista_x, tf.float32)
  return input_encode

  # Encode do x_train e x_test
x_train = encode_input(x_train)
x_test = encode_input(x_test)

In [0]:
# Criação do modelo de RNN
int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
embedded_sequences = tf.keras.layers.Embedding(encoder.vocab_size, 128)(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(qtd_dep, activation="softmax")(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_23 (Embedding)     (None, None, 128)         1581824   
_________________________________________________________________
conv1d_66 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_44 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_67 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_45 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_68 (Conv1D)           (None, None, 128)         165

In [0]:
# Compilando modelo e configurando o processo de treinamento
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])

# Treinando o modelo
history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Epoch 1/12
155/155 [==============================] - 2s 13ms/step - loss: 2.4878 - accuracy: 0.3923 - val_loss: 1.5009 - val_accuracy: 0.6438
Epoch 2/12
155/155 [==============================] - 2s 12ms/step - loss: 1.0759 - accuracy: 0.7509 - val_loss: 0.8354 - val_accuracy: 0.8142
Epoch 3/12
155/155 [==============================] - 2s 12ms/step - loss: 0.6049 - accuracy: 0.8632 - val_loss: 0.6467 - val_accuracy: 0.8573
Epoch 4/12
155/155 [==============================] - 2s 12ms/step - loss: 0.4023 - accuracy: 0.9104 - val_loss: 0.5475 - val_accuracy: 0.8878
Epoch 5/12
155/155 [==============================] - 2s 12ms/step - loss: 0.2852 - accuracy: 0.9339 - val_loss: 0.5280 - val_accuracy: 0.8891
Epoch 6/12
155/155 [==============================] - 2s 12ms/step - loss: 0.2197 - accuracy: 0.9478 - val_loss: 0.5092 - val_accuracy: 0.9000
Epoch 7/12
155/155 [==============================] - 2s 12ms/step - loss: 0.1732 - accuracy: 0.9592 - val_loss: 0.4741 - val_accuracy: 0.9114

In [0]:
# Testando a qualidade do modelo
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

69/69 [==============================] - 0s 2ms/step - loss: 0.5521 - accuracy: 0.9100
Test Loss: 0.5520514845848083
Test Accuracy: 0.9100409150123596


In [0]:
# Função para predição
def sample_predict(sample_pred_text):
  encoded_sample_pred_text = encoder.encode(sample_pred_text)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return (predictions)

In [0]:
# Predição dos departamentos no df2
exemplos = [x for x in df2['DESC_AJUSTADA']]
dep = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  predictions = sample_predict(exemplo)
  probabilities = [np.argmax(predictions[0])]
  dep.append(probabilities)
# Retornando os departamentos
new_dep = dep_encode.inverse_transform(dep)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
# Ajustando dados de testes com as predições
df2['DEPARTAMENTO'] = new_dep
df2.head(2)

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,sal rosa himalaia alho,MERCEARIA SALGADA
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,bola natal acasa,DECORAÇÃO


### Etapa 2 - CATEGORIA

In [0]:
# Incluindo departamento na descrição
df['DESC_AJUSTADA'] = df['DEPARTAMENTO'].str.lower() + ' ' + df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9].+', '', regex=True)
df2['DESC_AJUSTADA'] = df2['DEPARTAMENTO'].str.lower() + ' ' + df2['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')

# Função para retirar stop words
def tokenize_df(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words]
  new_text = TreebankWordDetokenizer().detokenize(stop)
  return new_text

# Eliminando as stop words
df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].apply(tokenize_df)
df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].apply(tokenize_df)

In [0]:
# Selecionando as unicas palavras da NOVA DESCRIÇÃO PARCEIRO
texto = df[['DESC_AJUSTADA']]
texto['TOKENIZED'] = texto['DESC_AJUSTADA'].apply(word_tokenize)
texto = list(texto.TOKENIZED)
lista = [item for sublista in texto for item in sublista]
lista = sorted(lista)
unicas = set(lista)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# Gerando encoder com o vocabulário das palavras
encoder = tfds.features.text.SubwordTextEncoder(vocab_list=unicas)
print('Vocabulary size(DESCRIÇÃO PARCEIRO): {}'.format(encoder.vocab_size))

# Encode departamado
cat = df['CATEGORIA']
cat_encode = LabelEncoder()
cat_encode.fit_transform(cat)

Vocabulary size(DESCRIÇÃO PARCEIRO): 12392


array([ 85, 124, 327, ..., 134, 107, 282])

In [0]:
# Definindo feature e target
x = df['DESC_AJUSTADA']
y = cat_encode.fit_transform(cat)

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)

# Encode do x_train e x_test
x_train = encode_input(x_train)
x_test = encode_input(x_test)

In [0]:
# Criação do modelo de RNN
int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
embedded_sequences = tf.keras.layers.Embedding(encoder.vocab_size, 128)(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(qtd_cat, activation="softmax")(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_24 (Embedding)     (None, None, 128)         1586176   
_________________________________________________________________
conv1d_69 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_70 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_47 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_71 (Conv1D)           (None, None, 128)         165

In [0]:
# Compilando modelo e configurando o processo de treinamento
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])

# Treinando o modelo
history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Epoch 1/12
155/155 [==============================] - 2s 13ms/step - loss: 3.7681 - accuracy: 0.2544 - val_loss: 2.2639 - val_accuracy: 0.4721
Epoch 2/12
155/155 [==============================] - 2s 12ms/step - loss: 1.6434 - accuracy: 0.6108 - val_loss: 1.1132 - val_accuracy: 0.7070
Epoch 3/12
155/155 [==============================] - 2s 12ms/step - loss: 0.8811 - accuracy: 0.7649 - val_loss: 0.7070 - val_accuracy: 0.8137
Epoch 4/12
155/155 [==============================] - 2s 12ms/step - loss: 0.5565 - accuracy: 0.8489 - val_loss: 0.5358 - val_accuracy: 0.8578
Epoch 5/12
155/155 [==============================] - 2s 12ms/step - loss: 0.3968 - accuracy: 0.8907 - val_loss: 0.4696 - val_accuracy: 0.8741
Epoch 6/12
155/155 [==============================] - 2s 12ms/step - loss: 0.2972 - accuracy: 0.9177 - val_loss: 0.4633 - val_accuracy: 0.8778
Epoch 7/12
155/155 [==============================] - 2s 12ms/step - loss: 0.2317 - accuracy: 0.9347 - val_loss: 0.4374 - val_accuracy: 0.8919

In [0]:
# Testando a qualidade do modelo
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

69/69 [==============================] - 0s 2ms/step - loss: 0.4821 - accuracy: 0.9010
Test Loss: 0.4821252226829529
Test Accuracy: 0.9009541273117065


In [0]:
# Predição das categorias no df2
exemplos = [x for x in df2['DESC_AJUSTADA']]
cat = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  predictions = sample_predict(exemplo)
  probabilities = [np.argmax(predictions[0])]
  cat.append(probabilities)
# Retornando os departamentos
new_cat = cat_encode.inverse_transform(cat)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
# Ajustando dados de testes com as predições
df2['CATEGORIA'] = new_cat
df2.head(2)

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,mercearia salgada sal rosa himalaia alho,MERCEARIA SALGADA,CONDIMENTO NATURAL
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,decoração bola natal acasa,DECORAÇÃO,ENFEITES DE NATAL


### Etapa 3 - SUB-CATEGORIA

In [0]:
# Incluindo departamento na descrição
df['DESC_AJUSTADA'] = df['DEPARTAMENTO'].str.lower() + ' ' + df['CATEGORIA'].str.lower() + ' ' + df['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9]+', '', regex=True)
df2['DESC_AJUSTADA'] = df2['DEPARTAMENTO'].str.lower() + ' ' + df2['CATEGORIA'].str.lower() + ' ' + df2['DESCRIÇÃO PARCEIRO'].str.lower().str.replace('[0-9]+', '', regex=True)

# Excluindo da descrição puntuação, informações julgadas irrelevantes para a classificação.
for x in pontuacao:
  df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].str.replace(x, ' ')
  df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].str.replace(x, ' ')

# Função para retirar stop words
def tokenize_df_sub(tokenized_words):
  tokenized_words = word_tokenize(tokenized_words)
  stop = [word for word in tokenized_words if word not in stop_words_sub]
  new_text = TreebankWordDetokenizer().detokenize(stop)
  return new_text

# Eliminando as stop words
df['DESC_AJUSTADA'] = df['DESC_AJUSTADA'].apply(tokenize_df_sub)
df2['DESC_AJUSTADA'] = df2['DESC_AJUSTADA'].apply(tokenize_df_sub)

In [0]:
# Selecionando as unicas palavras da NOVA DESCRIÇÃO PARCEIRO
texto = df[['DESC_AJUSTADA']]
texto['TOKENIZED'] = texto['DESC_AJUSTADA'].apply(word_tokenize)
texto = list(texto.TOKENIZED)
lista = [item for sublista in texto for item in sublista]
lista = sorted(lista)
unicas = set(lista)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
# Gerando encoder com o vocabulário das palavras
encoder = tfds.features.text.SubwordTextEncoder(vocab_list=unicas)
print('Vocabulary size(DESCRIÇÃO PARCEIRO): {}'.format(encoder.vocab_size))

# Encode departamado
sub = df['SUB-CATEGORIA']
sub_encode = LabelEncoder()
sub_encode.fit_transform(sub)

Vocabulary size(DESCRIÇÃO PARCEIRO): 14013


array([1850, 1224, 1687, ...,  808,  191, 1868])

In [0]:
# Definindo feature e target
x = df['DESC_AJUSTADA']
y = sub_encode.fit_transform(sub)

# Dividindo dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)

# Encode do x_train e x_test
x_train = encode_input(x_train)
x_test = encode_input(x_test)

In [0]:
# Criação do modelo de RNN
int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
embedded_sequences = tf.keras.layers.Embedding(encoder.vocab_size, 128)(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)
preds = layers.Dense(qtd_sub, activation="softmax")(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_25 (Embedding)     (None, None, 128)         1793664   
_________________________________________________________________
conv1d_72 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_73 (Conv1D)           (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_49 (MaxPooling (None, None, 128)         0         
_________________________________________________________________
conv1d_74 (Conv1D)           (None, None, 128)         165

In [0]:
# Compilando modelo e configurando o processo de treinamento
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=['accuracy'])

# Treinando o modelo
history = model.fit(x_train, y_train, epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(x_test, y_test))

Epoch 1/12
155/155 [==============================] - 2s 15ms/step - loss: 6.5287 - accuracy: 0.0529 - val_loss: 5.8634 - val_accuracy: 0.0990
Epoch 2/12
155/155 [==============================] - 2s 14ms/step - loss: 5.2097 - accuracy: 0.1841 - val_loss: 4.4280 - val_accuracy: 0.2962
Epoch 3/12
155/155 [==============================] - 2s 14ms/step - loss: 3.8628 - accuracy: 0.3588 - val_loss: 3.4737 - val_accuracy: 0.4525
Epoch 4/12
155/155 [==============================] - 2s 14ms/step - loss: 2.9931 - accuracy: 0.4769 - val_loss: 3.0126 - val_accuracy: 0.5152
Epoch 5/12
155/155 [==============================] - 2s 14ms/step - loss: 2.4295 - accuracy: 0.5567 - val_loss: 2.6763 - val_accuracy: 0.5738
Epoch 6/12
155/155 [==============================] - 2s 14ms/step - loss: 2.0218 - accuracy: 0.6120 - val_loss: 2.4456 - val_accuracy: 0.6134
Epoch 7/12
155/155 [==============================] - 2s 14ms/step - loss: 1.7310 - accuracy: 0.6571 - val_loss: 2.2976 - val_accuracy: 0.6243

In [0]:
# Testando a qualidade do modelo
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

69/69 [==============================] - 0s 3ms/step - loss: 1.9688 - accuracy: 0.7065
Test Loss: 1.968794822692871
Test Accuracy: 0.706497073173523


In [0]:
# Predição das categorias no df2
exemplos = [x for x in df2['DESC_AJUSTADA']]
sub = []
for exemplo in tqdm.notebook.tqdm(exemplos):
  predictions = sample_predict(exemplo)
  probabilities = [np.argmax(predictions[0])]
  sub.append(probabilities)
# Retornando os departamentos
new_sub = sub_encode.inverse_transform(sub)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
# Ajustando dados de testes com as predições
df2['SUB-CATEGORIA'] = new_sub
df2.head(2)

,0,DESCRIÇÃO PARCEIRO,DESC_AJUSTADA,DEPARTAMENTO,CATEGORIA,SUB-CATEGORIA
0,1,SAL ROSA HIMALAIA C/ ALHO 500G,mercearia salgada condimento natural sal rosa ...,MERCEARIA SALGADA,CONDIMENTO NATURAL,ALHO TRADICIONAL
1,2,JG BOLA NATAL ACASA C/17 DR/PR/BC R952,decoração enfeites natal bola natal acasa dr p...,DECORAÇÃO,ENFEITES DE NATAL,BOLA DE NATAL
